In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@3594d4dd

In [2]:
import spark.implicits._

case class Query(query_name: String, result: String, position: Int, rating: Int)

val queryData: Seq[(String, String, Int, Int)] = Seq(
  ("Dog", "Golden Retriever", 1, 5),
  ("Dog", "German Shepherd", 2, 5),
  ("Dog", "Mule", 200, 1),
  ("Cat", "Shirazi", 5, 2),
  ("Cat", "Siamese", 3, 3),
  ("Cat", "Sphynx", 7, 4)
)

val queriesDS = queryData.toDF("query_name", "result", "position", "rating").as[Query]

queriesDS.show()

+----------+----------------+--------+------+
|query_name|          result|position|rating|
+----------+----------------+--------+------+
|       Dog|Golden Retriever|       1|     5|
|       Dog| German Shepherd|       2|     5|
|       Dog|            Mule|     200|     1|
|       Cat|         Shirazi|       5|     2|
|       Cat|         Siamese|       3|     3|
|       Cat|          Sphynx|       7|     4|
+----------+----------------+--------+------+



import spark.implicits._


defined class Query
queryData: Seq[(String, String, Int, Int)] = List(
  ("Dog", "Golden Retriever", 1, 5),
  ("Dog", "German Shepherd", 2, 5),
  ("Dog", "Mule", 200, 1),
  ("Cat", "Shirazi", 5, 2),
  ("Cat", "Siamese", 3, 3),
  ("Cat", "Sphynx", 7, 4)
)
queriesDS: Dataset[Query] = [query_name: string, result: string ... 2 more fields]

We define query quality as:

The average of the ratio between query rating and its position.

We also define poor query percentage as:

The percentage of all queries with rating less than 3.

Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

| query_name | quality | poor_query_percentage |
|------------|---------|-----------------------|
| Dog        | 2.50    | 33.33                 |
| Cat        | 0.66    | 33.33                 |

In [3]:
queriesDS.withColumn(
        "bad",
        when(col("rating") < 3, 1).otherwise(0)
    )
    .groupBy("query_name")
    .agg(round(avg(col("rating") / col("position")), 2).as("quality"), 
        round(sum(col("bad")) * 100.0 / count(lit(1)), 2).as("poor_query_percentage")
    )
    .show()

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                33.33|
|       Cat|   0.66|                33.33|
+----------+-------+---------------------+



In [4]:
spark.stop